In [1]:
import os
import cv2
import numpy as np
from cvl.dataset import OnlineTrackingBenchmark
from cvl.trackers import MOSSETracker, DCFMOSSETracker
from cvl.lib import get_roi, resume_performance, get_arguments
import matplotlib.pyplot as plt
from copy import copy, deepcopy
import torch
from tqdm.notebook import tqdm as tqdm
import json

In [2]:
ds_path = "Mini-OTB/"

In [3]:
seq_list = os.listdir(ds_path)

In [4]:
dataset = OnlineTrackingBenchmark(ds_path)

# 1) Choose the tracker

In [5]:
TRACKER = "hog" # ["mosse", "resnet", "mobilenet", "alexnet", "vgg16", "hog"]

In [6]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [7]:
if TRACKER == "mosse":
    tracker = MOSSETracker()
    squared = False
    bigger = True
elif TRACKER in ["resnet", "mobilenet", "alexnet", "vgg16", "hog"]:
    tracker = DCFMOSSETracker(dev = device, features = TRACKER)
    squared = False
    bigger = True

In [8]:
results = {}

In [9]:
for idx, seq in enumerate(dataset):
    print(f"----------------------------Evaluating video named {seq.sequence_name}----------------------------------")
    bboxes = []
    for frame_idx, frame in enumerate(seq):
        image_color = frame['image']
        if TRACKER == "mosse":
            image = np.sum(image_color, 2) / 3 # grayscale
        else:
            image = np.transpose(np.float64(image_color), (2, 0, 1))
        if frame_idx == 0:
            bbox = copy(frame['bounding_box'])
            aaaa = bbox
            if bbox.width % 2 == 0:
                bbox.width += 1
            if bbox.height % 2 == 0:
                bbox.height += 1
            roi = get_roi(bbox, squared = squared, bigger = bigger) # get roi slightly bigger that bbox
            tracker.start(image, bbox, roi) # first frame approach
        else:
            tracker.detect(image)
            tracker.update()
        bboxes.append(copy(tracker.bbox))
    ious = dataset.calculate_per_frame_iou(idx, bboxes)
    print("mIoU:", np.mean(ious))
    results[idx] = np.mean(ious)

----------------------------Evaluating video named Basketball----------------------------------
mIoU: 0.1402644286322447
----------------------------Evaluating video named Biker----------------------------------
mIoU: 0.257459211129827
----------------------------Evaluating video named BlurBody----------------------------------
mIoU: 0.0940761227537807
----------------------------Evaluating video named BlurCar3----------------------------------
mIoU: 0.30857555767479666
----------------------------Evaluating video named Bolt----------------------------------
mIoU: 0.16929482175383814
----------------------------Evaluating video named Box----------------------------------
mIoU: 0.05739060600093931
----------------------------Evaluating video named CarScale----------------------------------
mIoU: 0.24172134895715822
----------------------------Evaluating video named Coke----------------------------------
mIoU: 0.08468898568718565
----------------------------Evaluating video named Coupon-

/home/tmr/Documents/PhD/WASP-courses/WASP/1_Learning_Feat_Rep/code-mod2/wasp-lfr-m2/Module_2/code/cvl/lib.py:39: RuntimeWarning: divide by zero encountered in double_scalars
  c = 255 / np.log(1 + np.max(img_cropped))
/home/tmr/Documents/PhD/WASP-courses/WASP/1_Learning_Feat_Rep/code-mod2/wasp-lfr-m2/Module_2/code/cvl/lib.py:40: RuntimeWarning: invalid value encountered in multiply
  log_image = c * (np.log(img_cropped + 1))


mIoU: 0.058437846506920826
----------------------------Evaluating video named FaceOcc1----------------------------------
mIoU: 0.8478119481424137
----------------------------Evaluating video named Human7----------------------------------
mIoU: 0.14379025179025176
----------------------------Evaluating video named Human9----------------------------------
mIoU: 0.24096942612208064
----------------------------Evaluating video named Ironman----------------------------------
mIoU: 0.08930551606948438
----------------------------Evaluating video named Jogging----------------------------------
mIoU: 0.17202546883335343
----------------------------Evaluating video named KiteSurf----------------------------------
mIoU: 0.3259867761971549
----------------------------Evaluating video named Liquor----------------------------------
mIoU: 0.3162532922565212
----------------------------Evaluating video named Man----------------------------------
mIoU: 0.8941404090657822
----------------------------Ev

In [10]:
result_file = f"../results/ious/experiments/1/1st_layer_bigger/hog.json"

In [11]:
with open(result_file, 'w') as fp:
    json.dump(results, fp)

# 2) Load results

In [12]:
with open(result_file) as fp:
    data = json.load(fp)

In [13]:
mIoU = 0

In [14]:
for k,v in data.items():
    mIoU += v

In [15]:
k 

'29'

In [16]:
mIoU / 30

0.26611533237818297